In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt 
%matplotlib inline
from copy import deepcopy 

In [ ]:
data = pd.read_csv("D:/Projects/Projects/Churn_Modelling/artifacts/data/validation/valid/train.csv")
df = deepcopy(data)
data.shape, df.shape 

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df.shape 

In [ ]:
df.head()

In [ ]:
unnecessory_features = ["RowNumber", "CustomerId", "Surname"] 
unnecessory_features

In [ ]:
df.drop(unnecessory_features, axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
target = 'Exited'
target

In [ ]:
for feature in df.columns:
    print(feature)
    print("-"*len(feature))
    print(f"samples:{df[feature].unique().tolist()[:5]}\ndtype:{df[feature].dtype}, count:{len(df[feature].unique())}, falling_category:{"numerical" if df[feature].dtype != "O" and len(df[feature].unique()) > 15 else "categorical"}")
    print("="*140)

In [ ]:
numerical_features = [feature for feature in df.columns if df[feature].dtype != "O" and len(df[feature].unique()) > 15 and feature != target]
categorical_features = [feature for feature in df.columns if feature not in numerical_features and feature != target]
print(f"numerical:{numerical_features}, count:{len(numerical_features)}")
print(f"categorical:{categorical_features}, count:{len(categorical_features)}")

In [ ]:
for feature in numerical_features:
    print(feature)
    print("-"*len(feature))
    print(f"samples:{df[feature].unique().tolist()[:5]}\ndtype:{df[feature].dtype}, count:{len(df[feature].unique())}, falling_category:{"numerical" if df[feature].dtype != "O" and len(df[feature].unique()) > 15 else "categorical"}")
    print("="*140)

In [ ]:
for feature in categorical_features:
    print(feature)
    print("-"*len(feature))
    print(f"samples:{df[feature].unique().tolist()[:5]}\ndtype:{df[feature].dtype}, count:{len(df[feature].unique())}, falling_category:{"numerical" if df[feature].dtype != "O" and len(df[feature].unique()) > 15 else "categorical"}")
    print("="*140)

In [ ]:
for feature in categorical_features:
    if df[feature].dtype == 'O':
        print(feature)
        print("-"*len(feature))
        print(f"samples:{df[feature].unique().tolist()[:5]}\ndtype:{df[feature].dtype}, count:{len(df[feature].unique())}, falling_category:{"numerical" if df[feature].dtype != "O" and len(df[feature].unique()) > 15 else "categorical"}")
        print("="*140)

In [ ]:
categorical_features = [feature for feature in df.columns if df[feature].dtype == 'O' and feature not in numerical_features]
categorical_features

In [ ]:
df['Exited'].value_counts()

In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler 
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline 

In [ ]:
numerical_pipeline = Pipeline([
    ('scaler', StandardScaler())
])
categorical_pipeline = Pipeline([
    ('encoder', OneHotEncoder())
])

In [ ]:
ct = ColumnTransformer([
    ('numerical_pipeline', numerical_pipeline, numerical_features),
    ('categorical_pipeline', categorical_pipeline, categorical_features)
], remainder='passthrough', n_jobs=-1, verbose=True, verbose_feature_names_out=True)

In [ ]:
transformed_data = ct.fit_transform(df)

In [ ]:
ct.get_feature_names_out()

In [ ]:
[name.split('__')[1] for name in ct.get_feature_names_out()]

In [ ]:
df = pd.DataFrame(transformed_data, columns=[name.split('__')[1] for name in ct.get_feature_names_out()])

# upsampling

In [ ]:
from sklearn.utils import resample 

In [ ]:
df[target].value_counts()

In [ ]:
{v:k for k, v in df[target].value_counts().to_dict().items()}[min({v:k for k, v in df[target].value_counts().to_dict().items()})]

In [ ]:
{v:k for k, v in df[target].value_counts().to_dict().items()}[max({v:k for k, v in df[target].value_counts().to_dict().items()})]

In [ ]:
df_majority = df[df[target]=={v:k for k, v in df[target].value_counts().to_dict().items()}[max({v:k for k, v in df[target].value_counts().to_dict().items()})]]
df_minurity = df[df[target]=={v:k for k, v in df[target].value_counts().to_dict().items()}[min({v:k for k, v in df[target].value_counts().to_dict().items()})]]

In [ ]:
df_majority.head()

In [ ]:
df_minurity.head()

In [ ]:
df_majority.shape, df_minurity.shape

In [ ]:
len(df_majority)

In [ ]:
df_minurity_resampled = resample(df_minurity, replace=True, n_samples=len(df_majority), random_state=42)

In [ ]:
df_minurity_resampled.shape

In [ ]:
df_majority.shape, df_minurity_resampled.shape

In [ ]:
type(df_minurity_resampled)

In [ ]:
df_minurity_resampled.head()

In [ ]:
df = pd.concat([df_majority, df_minurity_resampled], axis=0)
df.shape

In [ ]:
df[target].value_counts()

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
df.tail()